# Using Hubble's law and finding his constant

In this assignment, we would be trying to find out Hubble's constant from the data we have. 

### 1) Loading and plotting the data #


In [1]:
#First we will import our libraries

import numpy as np 
import matplotlib.pyplot as plt

#This code belew make our graphs interactive 

%matplotlib notebook

#Then we will load the data
#Unpack=True unpacks each column into a separate array

velocity, distance, distance_error = np.loadtxt("hubble_data.csv", delimiter = ',' ,unpack=True) 

In [2]:
#Now we plot our data 
#The error bar function includes the plot function 
#We add caps so that reading the graph is easier 

plt.figure()
plt.errorbar(velocity,distance,yerr=distance_error,capsize = 5,fmt='o')

#add gridlines, titles and axis lables

plt.grid(True)
plt.title('Distance against recessional velocity graph') 
plt.xlabel('velocity (km/s)')          
plt.ylabel('distance (Mpc)');

<IPython.core.display.Javascript object>

## 2) Linear regression using NumPy

Here we will try find the weighted and unweighted lines of best fit

In [3]:
# Now we will calculate the gradient and intercepts for weighted line of bestfit with its uncertainties 

# A straight line has a polynomial degree of 1

degree = 1

# To correctly take errors into account, we set weights with w=1/error & use cov='unscaled'

coeffs, errors = np.polyfit(velocity,distance,degree,w=1/distance_error,cov='unscaled')
m, c = coeffs                      # unpack gradient and intercept from output
dm, dc = np.sqrt(np.diag(errors))  # errors are sqrt of diagonal terms

# Present results with errors

print(f"m = {m} +- {dm}")
print(f"c = {c} +- {dc}")

m = 0.01574034318901381 +- 0.000827025757910185
c = -8.32585537874202 +- 7.37387236367192


In [4]:
# Now we do it for the unweighted fits 
# Fit straight line parameters and calculate errors
# Still using degree 1, but no weights

coeffs_uw, errors_uw = np.polyfit(velocity,distance,degree,cov=True)
m_uw, c_uw = coeffs_uw                      # unpack gradient and intercept from output
dm_uw, dc_uw = np.sqrt(np.diag(errors_uw))  # errors are sqrt of diagonal terms

# Present results with errors

print(f"m = {m_uw} +- {dm_uw}")
print(f"c = {c_uw} +- {dc_uw}")

m = 0.01597939731196561 +- 0.0002206588105827228
c = -11.624040905828377 +- 4.078725830535066


Now we write it out to an appropriate level of precision. Looking at the data file, the velocity has a precison of 5 decimal places, hence I believe its appropriate to give the gradient, intercept and errors to to 5 decimal places as well.

In [5]:
print(f"The gradient for the weighted line of best fit  is {m:.5f} Mpc/km/s +- {dm:.5f} Mpc/km/s")
print(f"The intercept is {c:.5f} Mpc +- {dc:.5f} Mpc")
print(f"The gradient for the unweighted line of best fit  is {m_uw:.5f} Mpc/km/s +- {dm_uw:.5f} Mpc/km/s")
print(f"The intercept is {c_uw:.5f} Mpc+- {dc_uw:.5f} Mpc")

The gradient for the weighted line of best fit  is 0.01574 Mpc/km/s +- 0.00083 Mpc/km/s
The intercept is -8.32586 Mpc +- 7.37387 Mpc
The gradient for the unweighted line of best fit  is 0.01598 Mpc/km/s +- 0.00022 Mpc/km/s
The intercept is -11.62404 Mpc+- 4.07873 Mpc


In [6]:
#Now we plot both of the lines of best fit on a graph 

#Set up line weighted line

xline = np.array([0,42500])
yline_rec  = m * xline + c

#Set up graph

plt.figure()
plt.title("Distance against recessional velocity graph")
plt.xlabel("Velocity (km/s)")
plt.ylabel("Distance (Mpc)")

#Plot data and lines
plt.errorbar(velocity,distance,yerr=distance_error,fmt='.', capsize = 4, label="data")
plt.plot(xline,yline_rec,'-.',label='best fit line')
plt.legend()

#Calculate coordinates to plot unweighted fit

yline_uw  = m_uw * xline + c_uw

#Plot data, lines and grid

plt.plot(xline,yline_uw,':',label='unweighted fit')
plt.legend();
plt.grid(True)

<IPython.core.display.Javascript object>

## 3) Linear regression from scratch

To calculate the gradient of the unweighted line of best fit, we use the equation:

$$m=\frac{\sum_{i=1}^n(x_i-\bar{x})y_i}{\sum_{i=1}^n(x_i-\bar{x})^2}$$

 
To calculate the intercept, we use: 
 
$$c=\bar{y}-m\bar{x}$$

Now we will attempt to to find both of these out manually 

In [7]:
#Find means

x_mean = np.sum(velocity) / len(velocity)
y_mean = np.sum(distance) / len(distance)

#Create equations for numerator and denominator, this will make it easier than doing one whole equation

numerator = np.sum((velocity - x_mean) * distance)
denominator = np.sum((velocity - x_mean) ** 2)

#Find gradient

gradient = numerator / denominator

#Find y-intercept

y_intercept = y_mean - gradient * x_mean

print(f'Gradient of unweighted line of best fit is {gradient}')
print(f'y_intercept of unweighted line of best fit is {y_intercept}')

Gradient of unweighted line of best fit is 0.015979397311965605
y_intercept of unweighted line of best fit is -11.624040905828451


To find the uncertainty of the gradient of the unweighted line of best fit, we use:

$$\Delta m = \sqrt{\frac{S}{(n-2)D}}$$

To find the uncertainty of the intercept, we use:


$$\Delta c = \sqrt{\left( \frac{1}{n}+\frac{\bar{x}^2}{D} \right)
\frac{S}{(n-2}}$$

where

$$S=\sum_{i=1}^n d_i^2$$

and

$$D=\sum_{i=1}^n (x_i-\bar{x})^2$$

and 

$$ d_i = y_i - y_\mathrm{line}(x_i) = y_i - mx_i -c $$


In [8]:
#Set up equation for sum of square of residuals of distances

S = np.sum((distance - gradient * velocity - y_intercept) ** 2)

#Set up equation for sum of the squares of the deviance of velocities 

D = np.sum((velocity - x_mean) ** 2)


#Set up our equation for uncertainty in gradient
#When using the 'len()' function, it doesn't matter which variable we use since the number of each variable is the same


delta_m = (S / ((len(velocity)-2) * D)) ** 0.5

#Set up our equation for uncertainty in y-intercept

delta_c = (( 1 / len(velocity) + ((x_mean) ** 2 ) / D ) * ( S / (len(velocity)-2))) ** 0.5

print(f'Uncertianty of gradient of unweighted line of best fit is {delta_m}')
print(f'Uncertainty of y-intercept of unweighted line of best fit is {delta_c}')

Uncertianty of gradient of unweighted line of best fit is 0.00022065881058272276
Uncertainty of y-intercept of unweighted line of best fit is 4.078725830535065


Now we attempt to find the gradient and y-intercept of the weighted line of best fit
To do this we use the equations[1]:

$$
m = \frac
{\sum_i w_i \sum_i w_i x_i y_i - \sum_i w_i x_i \sum_i w_i y_i}
{\sum_i w_i \sum_i w_i x_i^2 - \left(\sum_i w_i x_i\right)^2}
$$

and 

$$
c = \frac
{\sum_{i=1}^n w_i y_i - m\sum_{i=1}^n w_i x_i}
{\sum_{i=1}^n w_i}
$$

where

$$ w_i = \frac{1}{(\Delta y_i)^2} $$

In [9]:
#This is going to be a very strenous equation to write down, so we will break it up 

numerator_1   = np.sum( 1 / distance_error ** 2) * np.sum( velocity * distance / distance_error ** 2 )
numerator_2   = np.sum( velocity / distance_error ** 2) * np.sum( distance / distance_error ** 2)
denominator_1 = np.sum( 1 / distance_error ** 2 ) * np.sum( velocity ** 2 / distance_error ** 2)
denominator_2 = np.sum( velocity / distance_error ** 2) ** 2

#Now we can put them all into 1 equation

m_weighted = (numerator_1 - numerator_2) / (denominator_1 - denominator_2)

c_weighted =  (np.sum( distance / distance_error ** 2) - m_weighted * np.sum( velocity / distance_error ** 2)) / np.sum( 1 / distance_error ** 2)

print(f'Gradient of weighted line of best fit is {m_weighted}')
print(f'Y-intercept of weighted line of best fit is {c_weighted}')

Gradient of weighted line of best fit is 0.015740343189013805
Y-intercept of weighted line of best fit is -8.325855378742009


Now we find the uncertainty of the weighted lines. We can do this by using the equations:
    
$$
\Delta m = \sqrt{\frac{
\sum_i w_i
}{
\sum_i w_i \sum_i w_i x_i^2 - \left( \sum_i w_i x_i \right)^2
}}
$$

and

$$
\Delta c = \sqrt{\frac{
\sum_i w_i x_i^2
}{
\sum_i w_i \sum_i w_i x_i^2 - \left( \sum_i w_i x_i \right)^2
}}
$$

In [10]:
#Like before, we will try to break down the equation

numerator_w1   = np.sum( 1 / distance_error ** 2)
denominator_w1 = np.sum( 1 / distance_error ** 2 ) * np.sum( velocity ** 2 / distance_error ** 2)
denominator_w2 = np.sum( velocity / distance_error ** 2) ** 2
numerator_w2   = np.sum( velocity ** 2 / distance_error ** 2)

delta_m_weighted   = (numerator_w1 / (denominator_w1 - denominator_w2)) ** 0.5
delta_c_weighted = (numerator_w2 / (denominator_w1 - denominator_w2)) ** 0.5

print(f'Uncertainty of gradient of weighted line of best fit is {delta_m_weighted}')
print(f'Uncertainty of y-intercept of weighted line of best fit is {delta_c_weighted}')

Uncertainty of gradient of weighted line of best fit is 0.0008270257579101849
Uncertainty of y-intercept of weighted line of best fit is 7.373872363671918


In [11]:
#Now we round and print all our values

print(f'Gradient of unweighted line of best fit is {gradient:.5f}')
print(f'y_intercept of unweighted line of best fit is {y_intercept:.5f}')
print(f'Uncertianty of gradient of unweighted line of best fit is {delta_m:.5f}')
print(f'Uncertainty of y-intercept of unweighted line of best fit is {delta_c:.5f}')
print(f'Gradient of weighted line of best fit is {m_weighted:.5f}')
print(f'Y-intercept of weighted line of best fit is {c_weighted:.5f}')
print(f'Uncertainty of gradient of weighted line of best fit is {delta_m_weighted:.5f}')
print(f'Uncertainty of y-intercept of weighted line of best fit is {delta_c_weighted:.5f}')

Gradient of unweighted line of best fit is 0.01598
y_intercept of unweighted line of best fit is -11.62404
Uncertianty of gradient of unweighted line of best fit is 0.00022
Uncertainty of y-intercept of unweighted line of best fit is 4.07873
Gradient of weighted line of best fit is 0.01574
Y-intercept of weighted line of best fit is -8.32586
Uncertainty of gradient of weighted line of best fit is 0.00083
Uncertainty of y-intercept of weighted line of best fit is 7.37387


The results are the same as when you use python functions to calculate compared to manually calculate it. 
The advantage of using python functions is that it is quicker and easier to compute data. The disadvantage is there may not be a python function available for the calculation you are trying to do. 
The advantage of manually calculating is that you can further manipulate the equations into a form you want, if you wanted to calculate something else or if you wanted to take into account a certain variable or constant. The disadvantage however is that it is much longer to calculate data and large equations may be difficult to form using code. 

## 4) Goodness of fit

For both our weighted and unweighted fits, we will calculate the normalised residual for each of our points using the equation:

$$
\mathrm{Normalised\ residual} = \frac{d_i}{\Delta y_i}
$$


In [12]:
#Form equations for normalised residual for weighted and unweighted fits  

nr = (distance - m * velocity - c) / distance_error

nr_uw = (distance - m_uw * velocity - c_uw) / distance_error

#Now we plot these residuals

plt.figure()
plt.plot(velocity,nr,'-.',label='Weighted fit residual')
plt.plot(velocity,nr_uw,':',label='Unweighted fit residual')

#Add gridlines, titles and axis lables

plt.grid(True)
plt.title('Residuals of distances') 
plt.xlabel('Velocity')          
plt.ylabel('Residual');

<IPython.core.display.Javascript object>

Now we will find the reduced chi-squared statistic for both the weighted and unweighted fits. We can do this by using the equation:

$$    \chi_{reduced}^2 =  \frac{\sum_{i=1}^n (\frac{d_i}{\Delta y_i})^2}{k}             $$

Where 

$$ K = n - m $$

K is number of degrees of freedom, n is number of data points and m is number of fit parameters

In [13]:
#Now we form our equation for the reduced chi-squared for the unweighted fit 
#We have 2 fit parameters since we have a slope and intercept 

chi_reduced_weighted = np.sum(nr ** 2) / (len(velocity) - 2)

#Now we do it for unweighted fit 

chi_reduced_unweighted = np.sum(nr_uw ** 2) / (len(velocity) - 2) 

print(f'Chi-squared statistic for the weighted fit is {chi_reduced_weighted}')
print(f'Chi-squared statistic for the unweighted fit is {chi_reduced_unweighted}')

Chi-squared statistic for the weighted fit is 0.06331407689610873
Chi-squared statistic for the unweighted fit is 0.07731915333482722


The residuals are positive and negative. Overall they most likely add up to 0 which is normal. But the residuals do follow a pattern, almost like a u shape. This could mean that the fitted model is biased. The residuals are also very close and small compared to the data points which means the data is close to the plotted line. 
The reduced chi-squared values are much lower than one, which means the model overfits the data a lot. This means that the model could be extremely biased and inaccurate. 

## 5) The Hubble constant 

Now we actually finally find a value for Hubble's constant, using the equation $ v = H_0 d $

In [14]:
#We rearrange the equation into what our gradient represents 

# d/v = 1/H0 = gradient 
# H0 = 1/gradient

#Now we calculate H0 for weighted and unweighted fit 

h0_w = 1 / m_weighted 
h0_uw = 1 / gradient

#Now to find the errors

h0_w_error = delta_m_weighted / m_weighted * h0_w
h0_uw_error = delta_m / gradient *h0_uw 

print(f'Weighted fit is {h0_w:.5f} km/s/Mpc +- {h0_w_error:.5f}')
print(f'Unweighted fit is {h0_uw:.5f} km/s/Mpc +- {h0_uw_error:.5f}')
print(f'Weighted fit is {h0_w:.5f} km/s/Mpc +- {h0_w_error:.5f}')
print(f'Unweighted fit is {h0_uw:.5f} km/s/Mpc +- {h0_uw_error:.5f}')

Weighted fit is 63.53102 km/s/Mpc +- 3.33803
Unweighted fit is 62.58058 km/s/Mpc +- 0.86417
Weighted fit is 63.53102 km/s/Mpc +- 3.33803
Unweighted fit is 62.58058 km/s/Mpc +- 0.86417


The current values for Hubble's constant tested range from 70-75 Km/s/Mpc. This mean that the weighted line of best fit gave the more accuarate answer. 
Considering there is still a discrepancy of what the actual value of Hubble's constant is, it shows how much error's play importance for measuring values, especially for this case where there isnt one accepted value. There is a lack of precision in the experiment inherintly, so the error really affects the result. If we consider our positive errors though, we are able to reach closer to the true value of Hubble's constant. 
Our gradient, 1/H0, has units Mpc/km/s. If we convert the distances to the same unit, they would cancel and we would just be left with seconds,s. This represents the age of the universe.

## References 

[1] Dash L, and Waugh B. *Mid-Term Computational Data Analysis Assignment: Supplementary Jupyter NotebookFile .* [online] UCL: London; 2022 [Accessed 17 November 2022]. Available from: https://moodle.ucl.ac.uk/course/view.php?id=31674

[2] Llorente-Garcia I and Jones P. *Data Analysis and Statistics Booklet, Practical Physics and Computing 1: Module PHAS0007.* [online] UCL: London; 2021 [Accessed 23 September 2022]. Available from: https://moodle.ucl.ac.uk/mod/resource/view.php?id=4305408

Task 4.2 from student 22006797

Task 2.2 from student 22006797